In [1]:
!pip install nb_black nbdime ipython

In [8]:
# pip3 install git+https://github.com/linto-ai/whisper-timestamped

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload module is not an IPython extension.


UsageError: Line magic function `%autoreload` not found.


In [5]:
from urllib.parse import urlparse, parse_qs
import whisper_timestamped as whisper
from pydub import AudioSegment
from logger import logger
from pytube import YouTube

import whisper

import json
import os

In [6]:
output_path_youtube = "YoutubeAudios"
output_path_transcription = "transcriptions"

In [7]:
from pytube import YouTube
import json
import whisper  # Assuming you have a package or module named whisper
from logger import logger


class YouTubeTranscriber:
    def __init__(self, url, output_path_youtube, output_path_transcription):
        self.output_path_youtube = output_path_youtube
        self.yt = YouTube(url)
        self.transcription = None
        self.url = url
        self.filename_path = None
        self.output_path_transcription = output_path_transcription

    def extract_main_domain_and_video_id(self):
        parsed_url = urlparse(self.url)
        domain_parts = parsed_url.netloc.split(".")
        main_domain = domain_parts[-2] if len(domain_parts) >= 2 else None
        query_params = parse_qs(parsed_url.query)
        video_id = query_params.get("v", [None])[0]
        self.video_id = f"{main_domain}_{video_id}"

    def download_youtube(self):
        self.filename = f"{self.video_id}.mp3"

        audio_stream = self.yt.streams.filter(only_audio=True).first()

        audio_stream.download(
            output_path=self.output_path_youtube, filename=self.filename
        )
        logger.info(f"Audio downloaded to {self.output_path_youtube}/{self.filename}")

    def transcribe_audio(self, model_name, device):
        audio = whisper.load_audio(f"{self.output_path_youtube}/{self.filename}")
        model = whisper.load_model(model_name, device=device)
        self.transcription = whisper.transcribe(model, audio, word_timestamps=True)

    def write_to_json(self):
        with open(f"{self.output_path_transcription}/{self.video_id}.json", "w") as f:
            json.dump(self.transcription, f)
        logger.info(
            f"Transcription downloaded to {self.output_path_transcription}/{self.video_id}.json"
        )

    def merge_segments(self, num_to_merge):
        merged_segments = []
        segments = self.transcription["segments"]
        for i in range(0, len(segments), num_to_merge):
            merged_dict = {}
            slice_ = segments[i : i + num_to_merge]

            # Merging the 'text' fields
            merged_dict["text"] = " ".join(item["text"] for item in slice_)

            # Get the 'start' time from the first dictionary and the 'end' time from the last dictionary
            merged_dict["start"] = int(slice_[0]["start"])
            merged_dict["end"] = int(slice_[-1]["end"])

            merged_segments.append(merged_dict)

        self.transcription["merged_segments"] = merged_segments

    def run(self, num_to_merge=4, model_name="base", device="cpu"):
        logger.info("extract_main_domain_and_video_id")
        self.extract_main_domain_and_video_id()

        logger.info("download_youtube")
        self.download_youtube()

        logger.info("transcribe_audio")
        self.transcribe_audio(model_name=model_name, device=device)

        logger.info("merge_segments")
        self.merge_segments(num_to_merge)

        logger.info("write_to_json")
        self.write_to_json()


# Usage
output_path = output_path_youtube
url = "https://www.youtube.com/watch?v=5p248yoa3oE"
# url = "https://www.youtube.com/watch?v=UyoXmHS-KGc"
yt_transcriber = YouTubeTranscriber(
    url=url,
    output_path_youtube=output_path_youtube,
    output_path_transcription=output_path_transcription,
)

yt_transcriber.run()

2023-10-10 17:12:35,305 ./logs/auto-labeler INFO extract_main_domain_and_video_id [3050587144.py]
2023-10-10 17:12:35,307 ./logs/auto-labeler INFO download_youtube [3050587144.py]
2023-10-10 17:12:37,157 ./logs/auto-labeler INFO Audio downloaded to YoutubeAudios/youtube_5p248yoa3oE.mp3 [3050587144.py]
2023-10-10 17:12:37,157 ./logs/auto-labeler INFO transcribe_audio [3050587144.py]
/Users/saazizi/miniconda3/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2023-10-10 17:15:33,343 ./logs/auto-labeler INFO merge_segments [3050587144.py]
2023-10-10 17:15:33,346 ./logs/auto-labeler INFO write_to_json [3050587144.py]
2023-10-10 17:15:33,393 ./logs/auto-labeler INFO Transcription downloaded to transcriptions/youtube_5p248yoa3oE.json [3050587144.py]


# LangChain

In [98]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [99]:
# success = load_dotenv(find_dotenv())
# print(success)

In [100]:
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import ChatOpenAI

from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

## Summary

### Load transcript

In [101]:
import json

In [102]:
json_file="transcriptions/youtube_5p248yoa3oE.json"

In [103]:
with open (json_file, "r") as f:
    docs = json.load(f)

### LLM

In [104]:
llm = ChatOpenAI(temperature=0)

### Templates

In [105]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [106]:
map_prompt_template = """
                      Write a summary of this chunk of text that includes the main points and any important details.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])


combine_prompt_template = """
text: {text}
Your task is to produce both a concise, entity-dense summary and a bullet-point list of key entities or ideas from the following transcribed text of a webinar or conference. This should be done over five iterative rounds.

For each round, execute these steps:

- **Step 1**: Identify salient entities from the transcribed text that have not been covered in prior summaries or bullet-point lists. These can be pivotal quotes, crucial events, or key technical terms. Use semicolons to separate multiple entities.

- **Step 2**: Create a revised summary that incorporates all the details and entities from the previous summary, with the aim of keeping the summary as concise as possible while maintaining clarity and detail.

**Guidelines**:

- **Initial Summary**: Begin with a summary that appropriately reflects the length and complexity of the transcribed text. The focus should be on overarching themes.

- **Impactful Writing**: Revise each summary to improve its flow and coherence, making room for the inclusion of new important entities.

- **Efficient Language**: Utilize techniques like sentence fusion, idea compression, and the removal of redundant phrases to make the summary more streamlined.

- **Self-Containment**: Aim for each summary to stand alone, providing a comprehensive understanding without requiring access to the full transcript.

After the final round of summary revisions, list all the key entities or important ideas identified during the five rounds in bullet-point form:

**Key Entities and Important Ideas:**
- Bullet point 1
- Bullet point 2
- ...

Answer.

"""

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

### Map reduce and generate summary

In [107]:
map_reduce_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

In [108]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 20,
    # length_function = len,
    # is_separator_regex = False,
)
texts = text_splitter.create_documents([docs["text"]])
len(texts)

36

In [75]:
map_reduce_outputs = map_reduce_chain({"input_documents": texts}, return_only_outputs=True)
# map_reduce_outputs

In [55]:
print(map_reduce_outputs["output_text"])

**Round 1:**

**Entities:**
- Dr. Andrew Wu
- AI
- Electricity
- General purpose technology
- Opportunities

**Revised Summary:**
Dr. Andrew Wu, an expert in AI, compares AI to electricity, stating that it is a general purpose technology with the potential to change many aspects of our lives. In his talk, he will discuss the various opportunities that AI presents.

**Round 2:**

**Entities:**
- Supervised learning
- Generative AI

**Revised Summary:**
Dr. Andrew Wu highlights the importance of supervised learning and generative AI in the field of AI. He explains that supervised learning is widely used in applications such as image recognition, speech recognition, and recommendation systems, while generative AI focuses on creating new content based on patterns learned from existing data.

**Round 3:**

**Entities:**
- Large-scale supervised learning
- Large AI models
- Google Brain team

**Revised Summary:**
Over the last decade, large-scale supervised learning and the development of la

### Extract last summary, entities and ideas

In [58]:
# This is an LLMChain to write a review of a play given a synopsis.
template = """Extract the final summary, entities and ideas
{summary}
Answer:"""
prompt_template = PromptTemplate(input_variables=["summary"], template=template)
final_report = LLMChain(llm=llm, prompt=prompt_template, output_key="final_report")

In [59]:
final_report.run(map_reduce_outputs["output_text"])

'Dr. Andrew Wu discusses the potential of AI as a general purpose technology, comparing it to electricity. He emphasizes the importance of supervised learning and generative AI in the field of AI, particularly in applications such as image recognition, speech recognition, and recommendation systems. The development of large AI models and large neural networks has driven significant progress in AI, with the Google Brain team playing a key role in this advancement. The focus now is on adding generative AI to supervised learning, specifically in the field of text generation. Large language models have greatly advanced AI, with techniques like RHDF used to refine and improve AI systems. The workflow for supervised learning is crucial in utilizing large language models. Overall, AI presents numerous opportunities and has the potential to greatly impact various aspects of our lives.'

## Construct retreival

### Preprocess docs

In [109]:
from llama_index.llms import OpenAI
from langchain.schema import Document 


In [110]:
def get_documents_from_transcription(json_file, chunk_sizes):
    """
    Concatenates words from a given JSON data structure with maximum chunk sizes.

    Parameters:
        json_file (str): The path to the JSON file.
        chunk_sizes (list of int): The list of maximum number of words allowed in each concatenated chunk.

    Returns:
        list: A list of nodes. Each node contains concatenated text and metadata (start_time, end_time).

    """
    def add_document_to_chunks(word_list, start_time, end_time, chunks,chunk_size):
        """Helper function to add a new Document to chunks."""
        text = " ".join(word_list)
        doc = Document(page_content=text, metadata={'start': start_time, 
                                              "end": end_time,
                                             "chunk_size":chunk_size})
        chunks.append(doc)

    concatenated_chunks = []
    current_chunk = []
    current_word_count = 0
    current_start_time = None
    current_end_time = None

    # Load JSON data
    with open(json_file, "r") as f:
        json_data = json.load(f)


    if not isinstance(chunk_sizes, list):
        chunk_sizes = [chunk_sizes]
    for chunk_size in chunk_sizes:
        for segment in json_data["segments"]:
            add_document_to_chunks(word_list=[segment["text"]], 
                                   start_time=segment["start"], 
                                   end_time=segment["end"], 
                                   chunks=concatenated_chunks, 
                                   chunk_size=len(segment["tokens"])
                                  )
            for word_info in segment["words"]:
                if current_word_count + 1 > chunk_size:
                    add_document_to_chunks(current_chunk, current_start_time, current_end_time, concatenated_chunks, chunk_size)
                    current_chunk = []
                    current_word_count = 0
                    current_start_time = None

                current_chunk.append(word_info["word"].strip())
                current_word_count += 1

                if current_start_time is None:
                    current_start_time = word_info["start"]
                current_end_time = word_info["end"]

        if current_chunk:
            add_document_to_chunks(current_chunk, current_start_time, current_end_time, concatenated_chunks, len(segment["tokens"]))

    return concatenated_chunks


# Example usage
chunk_sizes = [128, 256, 512, 1024]
docs = get_documents_from_transcription(json_file="transcriptions/youtube_5p248yoa3oE.json", chunk_sizes=chunk_sizes)
len(docs)


1577

In [112]:
# Build a sample vectorDB
from langchain.vectorstores import Chroma
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# VectorDB
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=docs, embedding=embedding)

In [113]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "advice"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)
retriever_from_llm

MultiQueryRetriever(tags=None, metadata=None, retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], metadata=None, vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x1562c7d00>, search_type='similarity', search_kwargs={}), llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='You are an AI language model assistant. Your task is \n    to generate 3 different versions of the given user \n    question to retrieve relevant documents from a vector  database. \n    By generating multiple perspectives on the user question, \n    your goal is to help the user overcome some of the limitations \n    of distance-based similarity search. Provide these alternative \n    questions separated by newlines. Original question: {question}', template_format='f-string', validate_template=True), llm=ChatOpenAI(cache=None, ve

In [116]:
vectordb.as_retriever().get_relevant_documents(query=" Google brain team")

[Document(page_content=" AI of a lot of people, it turns out that there are a lot of subject matter experts in today's world", metadata={'chunk_size': 25, 'end': 1834.88, 'start': 1829.8}),
 Document(page_content=" AI of a lot of people, it turns out that there are a lot of subject matter experts in today's world", metadata={'chunk_size': 25, 'end': 1834.88, 'start': 1829.8}),
 Document(page_content=" AI of a lot of people, it turns out that there are a lot of subject matter experts in today's world", metadata={'chunk_size': 25, 'end': 1834.88, 'start': 1829.8}),
 Document(page_content=" AI of a lot of people, it turns out that there are a lot of subject matter experts in today's world", metadata={'chunk_size': 25, 'end': 1834.88, 'start': 1829.8})]

In [119]:
retriever_from_llm.get_relevant_documents(query="what are best IA projects or idea?")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


[Document(page_content="think we've always managed to help the companies build a strong technical team than partnering with subject matter experts often results in exciting new opportunities. And I want to share with you one other weird aspect of one of the weird lessons I've learned about, you know, building startups, which is I like to engage only when there's a concrete idea. And this runs counter to bother the advice you hear from the design thinking methodology, which often says don't rush to solutioning, right? Explore a lot of alternatives to avoid the solution. Honestly, we tried that. It was very slow. But what we've learned is that at the ideation stage, if someone comes to me and says, hey, Andrew, you should apply AI to financial services. Because I'm not a subject matter expert in financial services, it's very slow for me to learn enough about financial services that you can figure out what to do. I mean, eventually, you could get a good outcome, but it's a very labor inte